In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL=os.getenv('LLM_MODEL')

In [2]:
# For OpenAI after 1.0.0 (1.13.3 at the time of writing)
client = openai.OpenAI()

def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

![](../images/06-iterative.png)

![](../images/07-process.png)

## Generate a marketing product description from a product fact sheet

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your workspace with our exquisite mid-century inspired office chair, a versatile addition to our elegant collection that includes filing cabinets, desks, bookcases, meeting tables, and more. Designed to blend seamlessly into both home and business environments, this chair is not only a statement of style but also a testament to functionality.

Choose from a palette of shell colors and complement them with your preferred base finish—opt for the sleek stainless steel, the modern matte black, the pristine gloss white, or the classic chrome. Tailor the chair to your comfort and aesthetic preferences with options for plastic back and front upholstery (SWC-100) or indulge in the luxurious full upholstery (SWC-110), available in 10 fabric and 6 leather finishes.

Whether you're reaching for the highest shelf or sitting down to focus, the pneumatic chair adjustment offers a smooth raise/lower action, ensuring your comfort is never compromised. The chair's dimensions are meticulously cr

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [5]:
len(response)

2378

In [6]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Experience Italian craftsmanship with our mid-century inspired office chair, featuring a 10mm thick, PA6/PA66 nylon-coated cast aluminum shell and HD36 foam seat. Choose from plastic or fully upholstered options in 10 fabrics and 6 leathers, complemented by stainless steel, matte black, gloss white, or chrome bases. Optional armrests and caster types cater to your comfort and floor needs. Suitable for both home and contract use, with pneumatic height adjustment for optimal ergonomics.


Try to say "Use at most 3 sentences" or "Use at most 280 characters"

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [7]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Experience Italian craftsmanship with our mid-century inspired office chair, featuring a 10mm thick, PA6/PA66 nylon-coated cast aluminum shell and a 5-wheel aluminum base. Choose from plastic or fully upholstered in 10 fabrics and 6 leathers, with stainless steel, matte black, white gloss, or chrome finishes. Options include armrests and soft/hard-floor casters, ensuring comfort with HD36 foam and adjustable seat densities. Suitable for both home and contract use.


In [8]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your office space with our mid-century inspired chair, featuring a variety of shell colors and base finishes. Constructed with a durable 5-wheel coated aluminum base and pneumatic adjustment, this chair offers both style and functionality. Choose from plastic or fully upholstered options in premium fabrics or leather, complemented by stainless steel, matte black, gloss white, or chrome bases. Designed for versatility, it's available with or without armrests and caters to different preferences with soft or hard-floor casters and two foam densities. Made in Italy, this chair is a testament to quality and design. Product IDs: SWC-100, SWC-110.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

```html
<div>
    <p>Introducing our exquisite line of mid-century inspired office furniture, designed to elevate any workspace with its timeless elegance and modern functionality. This collection, which includes a range of filing cabinets, desks, bookcases, meeting tables, and more, is crafted to meet the demands of both home and business environments and is fully qualified for contract use.</p>
    <p>Our versatile chairs come in two distinct models: the SWC-100, featuring a sleek plastic back with front upholstery, and the SWC-110, offering full upholstery with a choice of 10 fabric and 6 leather options to complement any office decor. The base of the chair is available in four sophisticated finishes: stainless steel, matte black, gloss white, or chrome, allowing for a personalized touch.</p>
    <p>Constructed with a durable 5-wheel plastic coated aluminum base, the chair provides stability and ease of movement. The pneumatic adjust feature ensures effortless height customization f

## Load Python libraries to view HTML

In [10]:
from IPython.display import display, HTML

In [11]:
display(HTML(response))

Dimension,Measurement (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


## Try experimenting on your own!